In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, Imputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.model_selection._search import RandomizedSearchCV
from sklearn.feature_selection.variance_threshold import VarianceThreshold
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.tree.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
%matplotlib inline  
%load_ext autotime

In [2]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


time: 10.2 ms


In [3]:
# Load data and identify target column
df_australian = pd.read_csv("phpelnJ6y.csv")
target_column = 'A15'

time: 22.8 ms


In [4]:
# Consider a variable as nominal if there are less than 20 uniquie values (cutoff open to change)
nominal_vars = [name for name,value in df_australian.iteritems() if len(set(value)) < 20 and name != target_column]

# Get all object variables (categorical)
object_vars = [column[0] for column in df_australian.dtypes.iteritems() if column[1] == object and 
               column[0] != target_column]

# Get set of variables to encode to categorical
nominal_vars.extend(object_vars)
factors = set(nominal_vars)

# One hot encode each categorical variable as 0 or 1
df_australian = pd.get_dummies(df_australian, columns=factors)

time: 37.5 ms


In [5]:
# Initialize predictor variables
first_col = df_australian.columns[0]
X = np.array(df_australian[first_col])[:, np.newaxis]

# Get predictor variables into proper format
for variable in df_australian[df_australian.columns.difference([target_column, first_col])]:
    X = np.hstack((X, np.array(df_australian[variable])[:, np.newaxis]))
    
# Get target variable into proper format
le = LabelEncoder()
y = le.fit_transform(df_australian[target_column])

time: 19.9 ms


In [6]:
# Initialize pipeline steps
estimators = [('imputer', Imputer()), ('reduce_dim', PCA()), ('clf', AdaBoostClassifier())]

# Create pipeline
pipe = Pipeline(estimators)

# Pipeline parameters
pipe.get_params().keys()

dict_keys(['steps', 'imputer', 'reduce_dim', 'clf', 'imputer__axis', 'imputer__copy', 'imputer__missing_values', 'imputer__strategy', 'imputer__verbose', 'reduce_dim__copy', 'reduce_dim__iterated_power', 'reduce_dim__n_components', 'reduce_dim__random_state', 'reduce_dim__svd_solver', 'reduce_dim__tol', 'reduce_dim__whiten', 'clf__algorithm', 'clf__base_estimator', 'clf__learning_rate', 'clf__n_estimators', 'clf__random_state'])

time: 7.59 ms


In [7]:
# Number of predictor variables
num_feat = len(X[1])

# Parameters to search over
param_dist = {'clf__base_estimator': [DecisionTreeClassifier(), RandomForestClassifier(), 
                                      GradientBoostingClassifier(), ExtraTreesClassifier()],
             'clf__learning_rate': [.0001, .001, .01],
             'clf__n_estimators': [10, 50, 100, 250, 500], 
             'reduce_dim__n_components': np.arange(2,num_feat,int(num_feat/4))} # If 4 or more features

time: 3.1 ms


In [8]:
# Initialize random search to pipeline and parameters
n_iter = 10
ran_search = RandomizedSearchCV(estimator=pipe, param_distributions=param_dist, n_iter=n_iter)

# Fit random search to data
ran_search.fit(X,y)

# See results of search
report(ran_search.cv_results_, n_top=5)

Model with rank: 1
Mean validation score: 0.855 (std: 0.011)
Parameters: {'reduce_dim__n_components': 12, 'clf__n_estimators': 10, 'clf__learning_rate': 0.01, 'clf__base_estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)}

Model with rank: 2
Mean validation score: 0.841 (std: 0.012)
Parameters: {'reduce_dim__n_components': 12, 'clf__n_estimators': 10, 'clf__learning_rate': 0.001, 'clf__base_estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fr

In [11]:
# distribution of which ones have pipelines

time: 1.16 ms


In [9]:
flows = pd.read_csv("Results/OpenML Supervised Classification on blood-transfusion-service-center.csv")
ff = flows.groupby('Flow')["Flow"].count()
ff

Flow
classif.bartMachine(3)                                                                                                                                                                                                                                                                                                                                                                                                    1
mlr.classif.earth.preproc(1)                                                                                                                                                                                                                                                                                                                                                                                              1
mlr.classif.ranger(10)                                                                                                                                                     

time: 47.4 ms


In [10]:
flows = pd.read_csv("results/OpenML Supervised Classification on credit-g.csv")
ff = flows.groupby('Flow')["Flow"].count()
ff

Flow
classif.ada(5)                                                                                                                                                                                                                                                                                1
mlr.classif.ranger(13)                                                                                                                                                                                                                                                                     4743
mlr.classif.ranger(15)                                                                                                                                                                                                                                                                        7
mlr.classif.ranger(16)                                                                                                             

time: 35 ms


In [11]:
flows = pd.read_csv("results/OpenML Supervised Classification on Australian.csv")
ff = flows.groupby('Flow')["Flow"].count()
ff

Flow
mlr.classif.earth.preproc(1)                                                                                                                                                                                                                                                                                                                                                                                              1
mlr.classif.glmnet(11)                                                                                                                                                                                                                                                                                                                                                                                                  767
mlr.classif.glmnet(4)                                                                                                                                                      

time: 34.2 ms


In [12]:
flows

,Run ID,Flow,predictive_accuracy,Hyperparameters
0,4983858,mlr.classif.xgboost(9),0.8841,"eta:0243624191290608,depth:9,weight:5519373359..."
1,5176622,mlr.classif.xgboost(9),0.8826,"eta:0792272761741505,depth:4,weight:7356633752..."
2,4689191,mlr.classif.xgboost(9),0.8826,"eta:0630616261491632,depth:11,weight:271500836..."
3,5752214,mlr.classif.ranger(13),0.8826,"num.trees:19,mtry:14,sample.fraction:500674122..."
4,5940307,sklearn.pipeline.Pipeline(imputation=openmlstu...,0.8826,"weight:null,criterion:""gini"",depth:8,features:..."
5,5950605,sklearn.pipeline.Pipeline(imputation=openmlstu...,0.8826,"weight:null,criterion:""gini"",depth:7,features:..."
6,4349458,mlr.classif.xgboost(9),0.8812,"eta:0110750349439172,weight:25502013940564,sub..."
7,4470203,mlr.classif.ranger(13),0.8812,"num.trees:15,mtry:2,replace:FALSE,sample.fract..."
8,4595583,mlr.classif.xgboost(9),0.8812,"eta:114286117616671,depth:12,weight:2998373986..."
9,5851478,mlr.classif.xgboost(9),0.8812,"eta:051309794318711,depth:8,weight:04462212350..."


time: 66 ms
